In [3]:
import numpy as np
import matplotlib as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [41]:
def zero_padding(X, p):
    """padding X with zeros
    X - numpy array (batch_size, height, width, n_channels)
    p - how many zeros pad around X
    
    Returns:
    X_pad = numpy array (batch_size, height + 2p, width + 2p, n_channels)
    """
    X_pad = np.pad(X, [(0,0), (p,p), (p,p), (0,0)], 'constant')
                  
    return X_pad

In [42]:
a = np.random.randn(4,3,3,2)
a_p = zero_padding(a,2)

print("Initial shape - {}".format(a.shape))
print("After padding - {}".format(a_p.shape))


Initial shape - (4, 3, 3, 2)
After padding - (4, 7, 7, 2)


In [45]:
def conv_step(a_slice_prev, W,b):
    
    return np.sum(np.multiply(a_slice_prev,W)) + float(b)

In [47]:
np.random.seed(1)
a_slice_prev = np.random.randn(4, 4, 3)
W = np.random.randn(4, 4, 3)
b = np.random.randn(1, 1, 1)

Z = conv_step(a_slice_prev, W, b)
print("Z =", Z)

Z = -6.999089450680221


In [60]:
def conv_forward(A,W,b,hparameters):
    """Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    batch_size, height, width, n_channels = A.shape
    f,f,n_filters,n_channels = W.shape
    
    pad, stride = hparameters['pad'], hparameters['stride']
    A_pad = zero_padding(A,pad)
    
    result_height, result_width = int((height - f + 2 * pad) / stride) + 1, int((width - f + 2 * pad) / stride) + 1
    
    Z = np.zeros((batch_size,result_height, result_width,n_filters))
    
    for i in range(batch_size):
        a_p = A_pad[i,:,:,:]
        for j in range(result_width):
            for k in range(result_height):
                
                vert_start = k * stride
                horiz_start = j * stride
                
                vert_end = vert_start + f
                horiz_end = horiz_start + f
                
                a_slice_p = a_p[vert_start:vert_end, horiz_start:horiz_end,:]
                
                for filter_ in range( n_filters):
                    Z[i,k,j,filter_] = conv_step(a_slice_p, W[:,:,:,filter_],b[:,:,:,filter_])
                    
    cache = (A,W,b,hparameters)
    return Z, cache

In [61]:
np.random.seed(1)
A_prev = np.random.randn(10,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,1,8)
hparameters = {"pad" : 2,
               "stride": 2}

Z, cache_conv = conv_forward(A_prev, W, b, hparameters)
print("Z's mean =", np.mean(Z))
print("Z[3,2,1] =", Z[3,2,1])
print("cache_conv[0][1][2][3] =", cache_conv[0][1][2][3])

Z's mean = -0.1260359654114688
Z[3,2,1] = [-0.61490741 -6.7439236  -2.55153897]
cache_conv[0][1][2][3] = [-0.20075807  0.18656139  0.41005165]
